In [ ]:
import yfinance as yf  
import pandas as pd
import numpy as np

from src.data_extraction import print_nan_summary

In [ ]:
start_date = "2008-01-02"
end_date = '2022-12-31'

In [ ]:
from src.data_extraction import get_top_indian_stocks_tickers
from src.data_extraction import batch_download_price_data

tickers = get_top_indian_stocks_tickers()
df = batch_download_price_data(tickers, start_date, end_date)

In [ ]:
# df = Indian_stocks_df(start_date, end_date)

In [ ]:
df.to_csv("datas/raw_data.csv")

In [ ]:
print('The length of the raw data set: ',df.shape)
df.head()

In [ ]:
print_nan_summary(df)

In [ ]:
nan_summary = (
    df.isna()
      .sum()
      .loc[lambda x: x > 100]
      .to_frame(name="nan_count")
)
nan_summary

We see that there are many stocks that have empty data, we cannot remove them irresposibly because we will loose importamt information. We could start by keeping stocks that have atleast 90% of data available and then we could fill the empty data points with the previous values. But, we are carful here also to only fill atmost 3 consecutive data points with the prvious values, this is because if we are dealing withstocks that did not have an IPO before a time period or it got delisted after some time then filling without limit will cause us to have long periods of time with same price leading to wrong results.

In [ ]:
coverage = df.notna().mean()

keep_cols = coverage[coverage > 0.9].index
df = df[keep_cols]

df = df.ffill(limit=3)

print("The shape of the dataframe is:",df.shape)
print_nan_summary(df)


Okay apart from the data we removed we only have 13 columns with empty data points <= 10 % we could remove these stocks and do our analysis because we are still left with more than a 100 stocks which will still provide us with good insights without the unnessecary complexity

In [ ]:
nan_counts = df.isna().sum()
nan_cols = nan_counts[nan_counts >= 1]

print(nan_cols)

In [ ]:
df = df.dropna(axis=1)
print_nan_summary(df)

In [ ]:
print('The length of the cleaned data set: ',df.shape)
df.head()

In [ ]:
df.to_csv("datas/clean_data.csv")

We, want to work with the log returns because of it is approximately normally distributed and using the symmetric and additive property of log returns

In [ ]:
df = np.log(df/df.shift())
df.dropna()

In [ ]:
df.to_csv('datas/log_returns_data.csv')